# Sentiment Analysis dari Instagram comments calon Presiden 2024 sebelum dan sesudah deklrasi calon wakil presiden menggunakan Naive Bayes


## Tujuan : mengetahui perbedaan sentiment dari komentar instagram sebelum dan sesudah deklarasi calon wakil presiden


<ul>
    <li>
    Aurelius Ivan Wijaya (00000054769)
    </li>
    <li>
    Rajendra Abhinaya (00000060445)
    </li>
    <li>
    Maecyntha Irelynn Tantra (00000055038)
    </li>
    <li>
    Patricia theodora (00000054093)
    </li>
<ul>

# Data Collection

apa sih yang sebenernya kita cari?
* web scrapping algoritm untuk data primer (ivan)
* labeling (pat)
* data sekunder (mae)
* stopword library indonesia (abhi)
* cari jurnal referensi yang sudah, sebagai literature review (all, min 4 per person)

In [10]:
import pandas as pd
import numpy as np

# import dataset    
# [ Primary Dataset ]
anies_before = pd.read_csv('./Dataset/Anies/anies_before.csv', on_bad_lines='skip')
anies_after = pd.DataFrame(pd.read_csv('./Dataset/Anies/anies_after.csv', on_bad_lines="skip"))
ganjar_before = pd.DataFrame(pd.read_csv('./Dataset/Ganjar/ganjar_before.csv', on_bad_lines="skip"))
ganjar_after = pd.DataFrame(pd.read_csv('./Dataset/Ganjar/ganjar_after.csv', on_bad_lines="skip"))
prabowo_before = pd.DataFrame(pd.read_csv('./Dataset/Prabowo/prabowo_before.csv', on_bad_lines="skip"))
prabowo_after = pd.DataFrame(pd.read_csv('./Dataset/Prabowo/prabowo_after.csv', on_bad_lines="skip"   ))
# [ Secondary Dataset ]
instagram_cyber_comments = pd.read_csv('./Dataset/dataset_komentar_instagram_cyberbullying.csv')
tweet_tv = pd.read_csv('./Dataset/dataset_tweet_sentimen_tayangan_tv.csv')
tweet_pilkada = pd.read_csv('./Dataset/dataset_tweet_sentiment_pilkada_DKI_2017.csv')
tweet_opini_film = pd.read_csv('./Dataset/dataset_tweet_sentiment_opini_film.csv')
tweet_cellular = pd.read_csv('./Dataset/dataset_tweet_sentiment_cellular_service_provider.csv')
prastyo_sentiment = pd.read_csv('./Dataset/prastyo-sentiment_all.csv')
sentiment_twitter_indonesia = pd.read_csv('./Dataset/Indonesian Sentiment Twitter Dataset Labeled.csv', sep='\t')
# prastyo_sentiment_covid = pd.read_csv('./Dataset/prastyo-sentiment_posneg.csv')
display(tweet_pilkada.head())

,Id,Sentiment,Pasangan Calon,Text Tweet
0,1,negative,Agus-Sylvi,Banyak akun kloning seolah2 pendukung #agussil...
1,2,negative,Agus-Sylvi,#agussilvy bicara apa kasihan yaa...lap itu ai...
2,3,negative,Agus-Sylvi,Kalau aku sih gak nunggu hasil akhir QC tp lag...
3,4,negative,Agus-Sylvi,Kasian oh kasian dengan peluru 1milyar untuk t...
4,5,negative,Agus-Sylvi,Maaf ya pendukung #AgusSilvy..hayo dukung #Ani...


In [11]:
display(prabowo_after.head())

,index,comments,label
0,0,"Semangat, Pak.",1.0
1,1,BISMILLAH PAK ❤️ 🇮🇩,1.0
2,2,Gasss pak🔥,1.0
3,3,Ini yang kumau.,1.0
4,4,"Polling yuk, silakan tekan tombol yg pilih Ani...",1.0


## Data Integration

In [12]:
# 1: positive, 0: negative 

# data integration
# instagram_cyber_comments
# instagram_cyber_comments['label'] = 1
# change label name to 'comments'
instagram_cyber_comments.rename(columns={'Instagram Comment Text': 'comments'}, inplace=True)
# mapping sentiment
instagram_cyber_comments['Sentiment'] = instagram_cyber_comments['Sentiment'].map({'positive': 1, 'negative': 0})
instagram_cyber_comments['label'] = instagram_cyber_comments['Sentiment'].astype(int)
# drop unused columns
instagram_cyber_comments.drop(columns=['Id', 'Sentiment'], inplace=True)
# display(instagram_cyber_comments.head())
#change data type to string to ensure all data type is string
instagram_cyber_comments['comments'] = instagram_cyber_comments['comments'].astype(str)

# tweet_tv
# change label name to 'comments'
tweet_tv.rename(columns={'Tweet': 'comments'}, inplace=True)
# mapping sentiment
tweet_tv['Sentiment'] = tweet_tv['Sentiment'].map({'positive': 1, 'negative': 0})
tweet_tv['label'] = tweet_tv['Sentiment'].astype(int)
tweet_tv.rename(columns={'Text Tweet': 'comments'}, inplace=True)
tweet_tv.drop(columns=['Id', 'Sentiment', "Jumlah Retweet", "Acara TV"], inplace=True)
#change data type to string to ensure all data type is string
tweet_tv['comments'] = tweet_tv['comments'].astype(str)

# tweet_pilkada
# tweet_pilkada add new column 'label'
# tweet_pilkada['label'] = tweet_pilkada['Sentiment'].map({'positive': 1, 'negative': 0}).astype(int)
tweet_pilkada['Sentiment'] = tweet_pilkada['Sentiment'].map({'positive': 1, 'negative': 0}).astype(int)
# tweet_pilkada['label'] = tweet_pilkada['Sentiment'].astype(int)
tweet_pilkada = tweet_pilkada[['Sentiment', 'Text Tweet']]
tweet_pilkada.rename(columns={'Text Tweet': 'comments'}, inplace=True)
tweet_pilkada.rename(columns={'Sentiment': 'label'}, inplace=True)
#change data type to string to ensure all data type is string
tweet_pilkada['comments'] = tweet_pilkada['comments'].astype(str)

# tweet_opini_film
# change label name to 'comments'
tweet_opini_film.rename(columns={'Tweet': 'comments'}, inplace=True)
# mapping sentiment
tweet_opini_film['Sentiment'] = tweet_opini_film['Sentiment'].map({'positive': 1, 'negative': 0})
tweet_opini_film['label'] = tweet_opini_film['Sentiment'].astype(int)
tweet_opini_film.drop(columns=['Id', 'Sentiment'], inplace=True)
tweet_opini_film.rename(columns={'Text Tweet': 'comments'}, inplace=True)
#change data type to string to ensure all data type is string
tweet_opini_film['comments'] = tweet_opini_film['comments'].astype(str)

# tweet_cellular
# change label name to 'comments'
tweet_cellular.rename(columns={'Tweet': 'comments'}, inplace=True)
# mapping sentiment
tweet_cellular['Sentiment'] = tweet_cellular['Sentiment'].map({'positive': 1, 'negative': 0})
tweet_cellular['label'] = tweet_cellular['Sentiment'].astype(int)
tweet_cellular.drop(columns=['Id', 'Sentiment'], inplace=True)
tweet_cellular.rename(columns={'Text Tweet': 'comments'}, inplace=True)

# prastyo_sentiment
map = {'pos': 1, 'neg': 0, 'neu': 1}
prastyo_sentiment['label'] = prastyo_sentiment['label'].map(map)


# sentiment twitter indonesia
# change label name to 'comments'
sentiment_twitter_indonesia.rename(columns={'Tweet': 'comments'}, inplace=True)
# mapping sentiment
# sentiment_twitter_indonesia['sentimen'] = sentiment_twitter_indonesia['sentimen'].map({'positive': 1, 'negative': 0})
sentiment_twitter_indonesia['label'] = sentiment_twitter_indonesia['sentimen'].astype(int)
# integrate all secondary dataset
secondary_dataset = pd.concat([
    # prabowo_after,
    # prabowo_before,
    # ganjar_after,
    # ganjar_before,
    # sentiment_twitter_indonesia
    tweet_pilkada
    ], ignore_index=True)
#change data type to string to ensure all data type is string
secondary_dataset['comments'] = secondary_dataset['comments'].astype(str)

In [13]:
display(secondary_dataset.head())

,label,comments
0,0,Banyak akun kloning seolah2 pendukung #agussil...
1,0,#agussilvy bicara apa kasihan yaa...lap itu ai...
2,0,Kalau aku sih gak nunggu hasil akhir QC tp lag...
3,0,Kasian oh kasian dengan peluru 1milyar untuk t...
4,0,Maaf ya pendukung #AgusSilvy..hayo dukung #Ani...


# Pre-Proccessing

## Data Cleaning

### Handle Missing Value

In [14]:
def handleMissingValue(df):
    df = df.dropna()
    df = df.drop_duplicates()
    df = df[df['comments'] != '']
    # remove 'NaN' value
    if('NaN' in df['comments']):
        df = df[df['comments'] != 'NaN']
    df = df[df['comments'].notna()]
    df = df.reset_index(drop=True)
    df = df[['comments']] 
    return df
secondary_dataset['comments'] = handleMissingValue(secondary_dataset)

In [15]:
display(secondary_dataset.head())

,label,comments
0,0,Banyak akun kloning seolah2 pendukung #agussil...
1,0,#agussilvy bicara apa kasihan yaa...lap itu ai...
2,0,Kalau aku sih gak nunggu hasil akhir QC tp lag...
3,0,Kasian oh kasian dengan peluru 1milyar untuk t...
4,0,Maaf ya pendukung #AgusSilvy..hayo dukung #Ani...


In [16]:
print(secondary_dataset.isnull().sum())
# delete duplicate data and missing value and null value
secondary_dataset = secondary_dataset.dropna()
secondary_dataset = secondary_dataset.drop_duplicates()
secondary_dataset = secondary_dataset[secondary_dataset['comments'] != '']
# remove 'NaN' value
if('NaN' in secondary_dataset['comments']):
    secondary_dataset = secondary_dataset[secondary_dataset['comments'] != 'NaN']
secondary_dataset = secondary_dataset[secondary_dataset['comments'].notna()]
secondary_dataset = secondary_dataset.reset_index(drop=True)
# secondary_dataset = secondary_dataset[['comments']]
print(secondary_dataset.isnull().sum())

display(secondary_dataset.head())
# secondary_dataset.info()

label       0
comments    2
dtype: int64
label       0
comments    0
dtype: int64


,label,comments
0,0,Banyak akun kloning seolah2 pendukung #agussil...
1,0,#agussilvy bicara apa kasihan yaa...lap itu ai...
2,0,Kalau aku sih gak nunggu hasil akhir QC tp lag...
3,0,Kasian oh kasian dengan peluru 1milyar untuk t...
4,0,Maaf ya pendukung #AgusSilvy..hayo dukung #Ani...


### Case Folding

handle case folding to make sure all the words are in the same case (lowercase)

In [17]:
def case_folding(data):
    datatemps = []
    for i in range(0, len(data)):
        try:
            commentTemp = data['comments'][i]
            if isinstance(commentTemp, str):  # Check if the value is a string
                datatemps.append(commentTemp.lower())
        except KeyError as e:
            # print(f"KeyError at index {i}: {e}")
            pass  # skip the row if there is no comment
    datatemps = {'comments': datatemps}
    return pd.DataFrame(datatemps)

secondary_dataset['comments'] =  pd.DataFrame(case_folding(secondary_dataset))

display(secondary_dataset.head())

,label,comments
0,0,banyak akun kloning seolah2 pendukung #agussil...
1,0,#agussilvy bicara apa kasihan yaa...lap itu ai...
2,0,kalau aku sih gak nunggu hasil akhir qc tp lag...
3,0,kasian oh kasian dengan peluru 1milyar untuk t...
4,0,maaf ya pendukung #agussilvy..hayo dukung #ani...


### Punctuation & Number & Whitespace Removal

In [18]:
import string
import re
import pandas as pd

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

custom_dict = pd.read_csv('./Dataset/TextNormalization/customdict.csv')
abbreviations_dict = pd.read_csv('./Dataset/TextNormalization/kamus_singkatan.csv')

stopword_factory = StopWordRemoverFactory()
stopwords = stopword_factory.get_stop_words()
stopword_bahasa = pd.read_csv('./Dataset/TextNormalization/stopwordbahasa.csv')

stemmer_factory = StemmerFactory()
stemmer = stemmer_factory.create_stemmer()

def clean_text(comment):
    comment = str(comment)
    
    # Remove @user
    comment = re.sub("@[^\s]+", " ", comment)
    
    # Remove URL
    comment = re.sub(r"http\S+", " ", comment)

    # Remove hashtag
    comment = re.sub(r"#\S+", " ", comment)

    # Remove RT
    comment = re.sub(r"RT", " ", comment)

    # Remove non-ascii words and characters
    comment = re.sub(r'[^\x00-\x7F]+', ' ', comment)

    # Remove punctuation
    comment = re.sub(f"[{string.punctuation}]", " ", comment)
    
    # Remove extra whitespaces
    comment = re.sub(r'\s+', ' ', comment)
    
    # Remove single characters
    comment = re.sub(r'\s+[a-zA-Z]\s+', ' ', comment)
    
    # Remove numbers
    comment = re.sub(r'\d+', '', comment)
    
    # Remove tags
    comment = re.sub("&lt;/?.*?&gt;", " &lt;&gt; ", comment)
    
    # Remove special characters and digits
    comment = re.sub("(\\d|\\W)+", " ", comment)
    
    # If a letter is repeated more than 2 times, replace it with 1 times
    comment = re.sub(r'(.)\1+', r'\1\1', comment)

    # Remove abbreviations only when it is not preceded by a letter
    for u in abbreviations_dict['singkatan']:
        comment = re.sub(rf"(?<![a-zA-Z]){u}(?![a-zA-Z])", abbreviations_dict[abbreviations_dict['singkatan'] == u]['kepanjangan'].values[0], comment)

    # Change word to its base form
    # for u in custom_dict['word']:
    #     comment = comment.replace(u, custom_dict[custom_dict['word'] == u]['base'].values[0])

    # # Remove custom stopword custom dictionary
    # for u in custom_dict['word']:
    #     comment = comment.replace(u, ' ')
    # for u in stopword_bahasa['word']:
    #     comment = comment.replace(u, ' ')

    # # # Remove stopwords
    comment = ' '.join([word for word in comment.split() if word.lower() not in stopwords])
    
    # # Lemmatization or stemming
    # comment = ' '.join([stemmer.stem(word) for word in comment.split()])
    
    return comment.strip()


def remove_punct(data):
    data['comments'] = data['comments'].apply(clean_text)
    return data

# Call the function
cleaned_data = remove_punct(secondary_dataset)

# Save to CSV
cleaned_data.to_csv('./cleaned_data.csv', index=False)

secondary_dataset['comments'] = cleaned_data['comments']

In [19]:
display(cleaned_data)

,label,comments
0,0,banyak akun kloning pendukung mulai menyerang ...
1,0,bicara apa kasihan yaa lap air matanya wkwkwkwk
2,0,kalau sih nunggu hasil akhir qc nunggu motif c...
3,0,kasihan kasihan peluru milyar tiap rw mempan m...
4,0,maaf pendukung dukung diputaran
...,...,...
893,1,kali bapak lihat rspun tetap selfie oke hand
894,1,merangkul semua orang batas usia kelamin oke h...
895,1,jagoanku dibidang digital smiling face with su...
896,1,


In [20]:
# Smote
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer

# Instantiate SMOTE
smote = SMOTE()
# Assuming 'comments' is the name of the text column
comments = secondary_dataset['comments']
# Assuming 'label' is name of the target column
y = secondary_dataset['label']

# TF-IDF vectorization
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(comments)

# Apply SMOTE to data
X_smote, y_smote = smote.fit_resample(X_tfidf, y)

# See how much data is imbalanced from the label after SMOTE
print(pd.Series(y_smote).value_counts())

label
0    450
1    450
Name: count, dtype: int64


### Text Normalization / Noise Removal

* this one need research (ivan)
* slang word dataset that i used : https://github.com/nasalsabila/kamus-alay

* Contoh sebelum: "Para mahasiswa yang memperoleh nilai yang rendah dalam ujian tidak diizinkan untuk mengikuti ujian ulang."
* Contoh sesudah: "Mahasiswa yang memperoleh nilai rendah dalam ujian tidak diizinkan mengikuti ujian ulang."

In [21]:
import pandas as pd
indo_slang_word = pd.read_csv('./Dataset/TextNormalization/colloquial-indonesian-lexicon.csv')
indo_slang_word.head()

def replace_slang_word(doc,slang_word):
    for index in  range(0,len(doc)-1):
        index_slang = slang_word.slang==doc[index]
        formal = list(set(slang_word[index_slang].formal))
        if len(formal)==1:
            doc[index]=formal[0]
    return doc

# def text_normalization(data):
#     datatemps = []
#     for i in range(0, len(data)):
#         comment = data.iloc[i, 0]  # Access the 'comments' column in the DataFrame
#         comment = comment.split()
#         comment = replace_slang_word(comment,indo_slang_word)
#         comment = ' '.join(comment)
#         datatemps.append(comment)
#     return datatemps

def text_normalization(data):
    datatemps = []
    for i in range(0, len(data)):
        comment = str(data['comments'][i])  # Convert to string
        comment = comment.split()
        comment = replace_slang_word(comment, indo_slang_word)
        comment = ' '.join(comment)
        datatemps.append(comment)
    return datatemps

# Call the functions successively
# anies_before['comments'] = pd.DataFrame(text_normalization(anies_before))
# anies_after['comments'] = pd.DataFrame(text_normalization(anies_after))
# ganjar_before['comments'] = pd.DataFrame(text_normalization(ganjar_before))
# ganjar_after['comments'] = pd.DataFrame(text_normalization(ganjar_after))
# prabowo_before['comments'] = pd.DataFrame(text_normalization(prabowo_before))
# prabowo_after['comments'] = pd.DataFrame(text_normalization(prabowo_after))
secondary_dataset['comments'] = pd.DataFrame(text_normalization(secondary_dataset))

In [22]:
display(secondary_dataset.head())

,label,comments
0,0,banyak akun kloning pendukung mulai menyerang ...
1,0,bicara apa kasihan ya lap air matanya wkwkwkwk
2,0,kalau sih menunggu hasil akhir qc menunggu mot...
3,0,kasihan kasihan peluru milyar tiap rw mempan m...
4,0,maaf pendukung dukung diputaran


### Stopwords Removal

* this one need research (abhi)

In [23]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# download nltk
nltk.download('punkt')

def remove_stopwords(data):
    datatemps = []
    for i in range(0, len(data)):
        comment = data['comments']  # Access the 'comments' column in the DataFrame
        # print(comment)
        stop_words = set(stopwords.words('indonesian'))
        word_tokens = word_tokenize(comment[i])
        filtered_sentence = [w for w in word_tokens if not w in stop_words]
        filtered_sentence = []
        for w in word_tokens:
            if w not in stop_words:
                filtered_sentence.append(w)
        comment = ' '.join(filtered_sentence)
        datatemps.append(comment)
    return datatemps

# Call the functions successively
# anies_before['comments'] = pd.DataFrame(remove_stopwords(anies_before))
# anies_after['comments'] = pd.DataFrame(remove_stopwords(anies_after))
# ganjar_before['comments'] = pd.DataFrame(remove_stopwords(ganjar_before))
# ganjar_after['comments'] = pd.DataFrame(remove_stopwords(ganjar_after))
# prabowo_before['comments'] = pd.DataFrame(remove_stopwords(prabowo_before))
# prabowo_after['comments'] = pd.DataFrame(remove_stopwords(prabowo_after))
secondary_dataset['comments'] = pd.DataFrame(remove_stopwords(secondary_dataset))


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Aurel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [24]:
display(secondary_dataset.head())

,label,comments
0,0,akun kloning pendukung menyerang paslon opini ...
1,0,bicara kasihan ya lap air matanya wkwkwkwk
2,0,sih menunggu hasil qc menunggu motif cuitan pa...
3,0,kasihan kasihan peluru milyar rw mempan menang...
4,0,maaf pendukung dukung diputaran


### Stemming / Lemmatization

* this one need research (mae)


In [25]:
# # # import Sastrawi package
# # from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
# # # create stemmer
# # factory = StemmerFactory()
# # stemmer = factory.create_stemmer()

# # def stemming(data):
# #     datatemps = []
# #     for i in range(0, len(data)):
# #         comment = data['comments']  # Access the 'comments' column in the DataFrame
# #         # print(comment)
# #         comment = stemmer.stem(comment[i])
# #         datatemps.append(comment)
# #     return datatemps

# # # Call the functions successively
# # secondary_dataset['comments'] = pd.DataFrame(stemming(secondary_dataset))
# # display(anies_after.head())


# from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# # Create stemmer
# factory = StemmerFactory()
# stemmer = factory.create_stemmer()

# # Define the stemming function
# def stem_text(text):
#     if isinstance(text, str):
#         return stemmer.stem(text)
#     else:
#         return text

# # Convert 'comments' column to string and apply the stemming function
# secondary_dataset['comments'] = secondary_dataset['comments'].astype(str).apply(stem_text)

# # Display the updated DataFrame
# display(secondary_dataset.head())


In [26]:
# display(secondary_dataset.head())
display(secondary_dataset.head())

,label,comments
0,0,akun kloning pendukung menyerang paslon opini ...
1,0,bicara kasihan ya lap air matanya wkwkwkwk
2,0,sih menunggu hasil qc menunggu motif cuitan pa...
3,0,kasihan kasihan peluru milyar rw mempan menang...
4,0,maaf pendukung dukung diputaran


## Tokenization

* this one need research (mae)

In [27]:
# def tokenization(data):
#     datatemps = []
#     for i in range(0, len(data)):
#         comment = data['comments']  # Access the 'comments' column in the DataFrame
#         # print(comment)
#         comment = word_tokenize(comment[i])
#         datatemps.append(comment)
#     return datatemps
def tokenization(data):
    datatemps = []
    for i in range(0, len(data)):
        comment = data['comments'][i]  # Access the 'comments' column in the DataFrame
        comment = word_tokenize(comment)
        datatemps.append(comment)
    return datatemps

# # Call the functions successively
# # anies_before['comments'] = pd.DataFrame(tokenization(anies_before))
# # anies_after['comments'] = pd.DataFrame(tokenization(anies_after))
# # ganjar_before['comments'] = pd.DataFrame(tokenization(ganjar_before))
# # ganjar_after['comments'] = pd.DataFrame(tokenization(ganjar_after))
# # prabowo_before['comments'] = pd.DataFrame(tokenization(prabowo_before))
# # prabowo_after['comments'] = pd.DataFrame(tokenization(prabowo_after))
# # secondary_dataset['comments'] = pd.DataFrame(tokenization(secondary_dataset))   

# anies_before['comments'] = tokenization(anies_before)
# anies_after['comments'] = tokenization(anies_after)
# ganjar_before['comments'] = tokenization(ganjar_before)
# ganjar_after['comments'] = tokenization(ganjar_after)
# prabowo_before['comments'] = tokenization(prabowo_before)
# prabowo_after['comments'] = tokenization(prabowo_after)
# secondary_dataset['comments'] = tokenization(secondary_dataset)

from sklearn.feature_extraction.text import CountVectorizer

# # CountVectorizer
vectorizer = CountVectorizer()
secondary_dataset_vectorizer = tokenization(secondary_dataset)
# print(vectorizer.get_feature_names())
# # display(X_train.shape)
# print(secondary_dataset['comments'])
# display(secondary_dataset)
display(secondary_dataset_vectorizer)
secondary_dataset['comments'] = secondary_dataset_vectorizer

[['akun',
  'kloning',
  'pendukung',
  'menyerang',
  'paslon',
  'opini',
  'argumen',
  'pmbenaran',
  'terkecoh'],
 ['bicara', 'kasihan', 'ya', 'lap', 'air', 'matanya', 'wkwkwkwk'],
 ['sih',
  'menunggu',
  'hasil',
  'qc',
  'menunggu',
  'motif',
  'cuitan',
  'pasca',
  'nyungsep'],
 ['kasihan',
  'kasihan',
  'peluru',
  'milyar',
  'rw',
  'mempan',
  'menangin',
  'pilkada'],
 ['maaf', 'pendukung', 'dukung', 'diputaran'],
 ['aneh',
  'deh',
  'lebay',
  'sangkut',
  'pautkan',
  'kandidat',
  'calgub',
  'lebay',
  'dukung'],
 ['allah', 'swt', 'ucapkan', 'syukur', 'terimakasih', 'makna', 'suara'],
 ['terima', 'kasih', 'teruntuk', 'pendukung', 'kalianlah', 'pembeda', 'kali'],
 ['terima', 'kasih', 'keistiqomahan', 'relawan', 'mengantarkan', 'signifikan'],
 ['mengenang', 'pidato', 'kekalahan'],
 ['dammnn', 'politik', 'cantik', 'ngorbanin'],
 ['batal', 'nyoblos', 'no', 'kertas', 'suara'],
 ['terbahak-bahak', 'survey', 'mu', 'panggang', 'api'],
 ['kemarin', 'april', 'suaraku', 'pa

In [28]:
display(secondary_dataset.head())


,label,comments
0,0,"[akun, kloning, pendukung, menyerang, paslon, ..."
1,0,"[bicara, kasihan, ya, lap, air, matanya, wkwkw..."
2,0,"[sih, menunggu, hasil, qc, menunggu, motif, cu..."
3,0,"[kasihan, kasihan, peluru, milyar, rw, mempan,..."
4,0,"[maaf, pendukung, dukung, diputaran]"


## Synthetic Minority Oversampling Technique (SMOTE)

* this one need research (ivan)
* Smote adalah sebuah tehnik yang digunakan terhadap data yang tidak seimbang

## K-Fold Cross Validation

* this one need research (abhi)

In [29]:
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Embedding, LSTM, Dense
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences

# # Assuming 'comments' is the text data in your dataset
# X = secondary_dataset['comments']
# y = secondary_dataset['label']

# # Encode the labels
# label_encoder = LabelEncoder()
# y = label_encoder.fit_transform(y)

# # Tokenize the text data
# max_words = 5000  # Maximum number of words to keep
# tokenizer = Tokenizer(num_words=max_words)
# tokenizer.fit_on_texts(X)
# X_sequences = tokenizer.texts_to_sequences(X)

# # Pad sequences to a fixed length (you can adjust maxlen as needed)
# maxlen = 100  # Example: Maximum length of a sequence
# X_padded = pad_sequences(X_sequences, maxlen=maxlen)

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)

# # Build the LSTM model
# model = Sequential()
# model.add(Embedding(input_dim=max_words, output_dim=128, input_length=maxlen))
# model.add(LSTM(128))
# model.add(Dense(1, activation='sigmoid'))

# # Compile the model
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# # Train the model
# model.fit(X_train, y_train, epochs=12, batch_size=32, validation_split=0.2)

# # Evaluate the model on the test set
# loss, accuracy = model.evaluate(X_test, y_test)
# print(f"\nTest Accuracy: {accuracy}")

# # Predictions
# predictions = model.predict(X_test)

In [33]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
# svc 
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
# xgboost
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Assuming 'comments' is the text data in your dataset
X = secondary_dataset['comments']
y = secondary_dataset['label']

# Join the list of strings into a single string for each document
X = [' '.join(comment) for comment in X]

# Use CountVectorizer to convert text data to a format suitable for RandomForestClassifier
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(X).toarray()  # Convert to array

# Define 10-fold cross-validation test harness
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Instantiate your classifier
classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Use cross_val_score to perform K-Fold Cross Validation
cv_results = cross_val_score(classifier, X, y, cv=kfold)

# Print the results for each fold
best = 0

for i, accuracy in enumerate(cv_results):
    if accuracy > best:
        best = i
        print(f"Fold {i + 1}: Accuracy = {accuracy} <== BEST RESULT")
    else:
        pass
    print(f"Fold {i + 1}: Accuracy = {accuracy}")
# Print the mean and standard deviation of the cross-validation results
print(f"\nMean Accuracy: {cv_results.mean()}")
print(f"Standard Deviation: {cv_results.std()}")

Fold 1: Accuracy = 0.6333333333333333 <== BEST RESULT
Fold 1: Accuracy = 0.6333333333333333
Fold 2: Accuracy = 0.5888888888888889 <== BEST RESULT
Fold 2: Accuracy = 0.5888888888888889
Fold 3: Accuracy = 0.6333333333333333
Fold 4: Accuracy = 0.6777777777777778
Fold 5: Accuracy = 0.7
Fold 6: Accuracy = 0.6555555555555556
Fold 7: Accuracy = 0.6333333333333333
Fold 8: Accuracy = 0.5555555555555556
Fold 9: Accuracy = 0.6292134831460674
Fold 10: Accuracy = 0.6067415730337079

Mean Accuracy: 0.6313732833957553
Standard Deviation: 0.03952714071212149
Score: 0.9944320712694877


In [41]:
# train test split
from sklearn.model_selection import train_test_split

# Assuming 'comments' is the text data in your dataset
X = secondary_dataset['comments']
X = [' '.join(comment) for comment in X]
X = vectorizer.fit_transform(X).toarray()  # Convert to array
X = pd.DataFrame(X)
y = secondary_dataset['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"\nTest Accuracy: {accuracy}")
display(X_train)


Test Accuracy: 0.6622222222222223


,0,1,2,3,4,5,6,7,8,9,...,2478,2479,2480,2481,2482,2483,2484,2485,2486,2487
377,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
357,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
723,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
306,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
464,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
270,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
860,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
435,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
from sklearn.model_selection import GridSearchCV

# Hyperparameter tuning for RandomForestClassifier
model_param_grid = {'alpha': [0.1, 0.5, 1.0, 2.0, 5.0, 10.0]}
model_grid_search = GridSearchCV(MultinomialNB(), model_param_grid, cv=kfold)
model_grid_search.fit(X, y)

# Print the best parameters and corresponding accuracy
print("\nBest parameters for RandomForestClassifier:", model_grid_search.best_params_)
print("Best accuracy for RandomForestClassifier:", model_grid_search.best_score_)


Best parameters for RandomForestClassifier: {'alpha': 5.0}
Best accuracy for RandomForestClassifier: 0.6892509363295879


In [32]:
# for LSTM

# Define the prediction function
def prediction(data):
    # Tokenize the input data
    tokenized = tokenizer.texts_to_sequences(data)
    
    # Pad sequences to the same length as in training
    padded = pad_sequences(tokenized, maxlen=maxlen)
    
    # Make predictions using the trained model
    predictions = model.predict(padded)

    # Convert predictions to labels
    labels = (predictions > 0.5).astype(int)
    
    return label_encoder.inverse_transform(labels)

# Call the function successively
text = ['anak baik']
pred = prediction(text)

print(pred)

NameError: name 'tokenizer' is not defined

In [ ]:
# text = ['anak bego']
# pred = prediction(text)
# print(pred)

In [ ]:
import pandas as pd
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer

# Assuming 'comments' is the text data in your dataset
X = pd.read_csv('./detik_anies_after.csv')
display(X.head())


def prediction(data):
    tokenized = []
    for i in range(0, len(data)):
        # Check if the value is NaN or not a string
        if pd.notna(data[i]) and isinstance(data[i], str):
            tokenized.extend(word_tokenize(data[i]))

    # Transform using the same vectorizer used during training
    text = vectorizer.transform([" ".join(tokenized)]).toarray()

    return classifier.predict(text)

pos = 0
neg = 0
for i in range(0, len(X)):
    text = [X['comments'][i]]
    pred = prediction(text)
    if pred == 1:
        pos += 1
    else:
        neg += 1
    

print(pos)
print(neg)


,index,comments
0,0,"Bismillah Pak Prabowo, plisss gandeng Pak Mahf..."
1,1,Lumayan buat lawakan pilpres nih 😂
2,2,"Inget kata2 Mbak Yenny Wahid, Gus Dur yang di ..."
3,3,Ternyata anis cuman buat pemecah suara aja 🤣😅....
4,4,Bagaimana kalau sudah menjadi penguasa! Dukung...


218
3


In [ ]:
import pandas as pd

# Assuming 'comments' is the text data in your dataset
X = pd.read_csv('./detik_anies_after.csv')
display(X.head())

pos = 0
neg = 0
for i in range(0, len(X)):
    text = [X['comments'][i]]
    display(text)
    pred = prediction(text)
    if pred == 1:
        pos += 1
    else:
        neg += 1



,index,comments
0,0,"Bismillah Pak Prabowo, plisss gandeng Pak Mahf..."
1,1,Lumayan buat lawakan pilpres nih 😂
2,2,"Inget kata2 Mbak Yenny Wahid, Gus Dur yang di ..."
3,3,Ternyata anis cuman buat pemecah suara aja 🤣😅....
4,4,Bagaimana kalau sudah menjadi penguasa! Dukung...


['Bismillah Pak Prabowo, plisss gandeng Pak Mahfud MD 😢😢👏👏']

['Lumayan buat lawakan pilpres nih 😂']

['Inget kata2 Mbak Yenny Wahid, Gus Dur yang di katakan oleh dia Adalah Guru Politik nya palah di Kudeta ,apa lagi Rakyat😂😂']

['Ternyata anis cuman buat pemecah suara aja 🤣😅..sayang sekali gk bisa dijadikan alternatif pilihan']

...

['Sudah dipastikan kalah😂🙌 congrat']

['Cak Imin bernafsu sekali, dari latar belakang apalagi yg katanya dekat sama kyai saya kira sudah bisa hidup tentram dan nyaman, eh malah bernafsu ingin jabatan apa lagi prosesnya bikin pusing dan cape']

['Disayangkan sekali sih. Skip dulu.']

['Dumb & Dumberr 😂']

['Pasangan AMIS...ANIES BASWEDAN - MUHAIMIN ISKANDAR😂']

In [ ]:
# display(ganjar_after.head())
# # ganjar_after['comments'] = tokenization(ganjar_after)
# negative = 0
# positive = 0
# for i in range(0, len(ganjar_after)):
#     pred = prediction(ganjar_after['comments'][i])
#     print(pred, ganjar_after['comments'][i])
#     if pred == 0:
#         negative += 1
#     else:
#         positive += 1
# print(negative, positive)


## Workload Distribution?

# Modeling

<!-- * kemungkinan Binomial Naive Bayes -->

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.naive_bayes import GaussianNB

# # Assuming X_train is a list of strings (or can be converted to strings)
# text_data = [str(item) for item in X_train]

# # Create a TfidfVectorizer
# vectorizer = TfidfVectorizer(min_df=2)  # Adjust min_df as needed

# # Vectorize the training set
# X_train_vectorized = vectorizer.fit_transform(text_data)

# # Assuming X_test is a list of strings (or can be converted to strings)
# X_test = [str(item) for item in X_test]

# # Train the Gaussian Naive Bayes model
# gnb = GaussianNB()
# gnb.fit(X_train_vectorized.toarray(), y_train)  # Convert sparse matrix to dense array

# # Vectorize the test set
# X_test_vectorized = vectorizer.transform(X_test)

# # Make predictions on the test set
# predictions = gnb.predict(X_test_vectorized.toarray())  # Convert sparse matrix to dense array

# # Continue with evaluation as needed


In [ ]:
# # accuracy
# from sklearn.metrics import accuracy_score
# X_test_vectorized = vectorizer.transform(X_test)
# y_pred = gnb.predict(X_test_vectorized.toarray())
# print('Accuracy: ', accuracy_score(y_test, y_pred))

In [ ]:
# # Multinomial Naive Bayes
# from sklearn.naive_bayes import MultinomialNB

# # Train the model
# mnb = MultinomialNB(alpha=0.8, class_prior=None, fit_prior=True)
# mnb.fit(X_train_vectorized, y_train)

In [ ]:
# # accuracy
# from sklearn.metrics import accuracy_score
# y_pred = mnb.predict(X_test_vectorized.toarray())
# print('Accuracy: ', accuracy_score(y_test, y_pred))

# # 0.7681818181818182

In [ ]:
# # Bernoulli Naive Bayes
# from sklearn.naive_bayes import BernoulliNB

# # Train the model
# bnb = BernoulliNB(alpha=1, class_prior=None, fit_prior=True)
# bnb.fit(X_train_vectorized, y_train)

In [ ]:
# # accuracy
# from sklearn.metrics import accuracy_score
# y_pred = bnb.predict(X_test_vectorized.toarray())
# print('Accuracy: ', accuracy_score(y_test, y_pred))

### Random Forest Classifier

In [ ]:
# # Random Forest Classifier

# from sklearn.ensemble import RandomForestClassifier

# # Train the model
# rfc = RandomForestClassifier(n_estimators=100, random_state=42)
# rfc.fit(X_train_vectorized, y_train)

# # accuracy
# from sklearn.metrics import accuracy_score
# y_pred = rfc.predict(X_test_vectorized.toarray())

# print('Accuracy: ', accuracy_score(y_test, y_pred))

In [ ]:
# # SVM Classifier
# from sklearn.svm import SVC

# # Train the model
# svm = SVC()

# svm.fit(X_train_vectorized, y_train)

# # accuracy

# from sklearn.metrics import accuracy_score
# y_pred = svm.predict(X_test_vectorized.toarray())
# print('Accuracy: ', accuracy_score(y_test, y_pred))

# Model Evaluation

* this one need research (pat)

# Analysis the data from primary data

In [ ]:
# def predict_sentiment(model, vectorizer, text):
#     # Vectorize the input text
#     text_vectorized = vectorizer.transform([text])
#     # Make predictions
#     prediction = model.predict(text_vectorized.toarray())
    
#     return prediction[0]

# # Test the model
# # print(predict_sentiment(gnb, vectorizer, text))
# # anies before

# def predict_batch(data):
#     positive_count = 0
#     negative_count = 0
#     for i in range(0, len(data)):
#         prediction = predict_sentiment(gnb, vectorizer, data['comments'][i])
#         if prediction == 1:
#             positive_count += 1
#         else:
#             negative_count += 1
#         # print('sentiment', prediction, data['comments'][i])
#     print('komentar positive: ', positive_count)
#     print('komentar negative: ', negative_count)


# predict_batch(ganjar_before)

## Modal Comprarison

# Conclusion